# Interactive time-series geospatial data on a web map <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[ga_ls_fc_3](https://explorer.sandbox.dea.ga.gov.au/products/ga_ls_fc_3)

## Background

[Leaflet](https://leafletjs.com/) is the leading open-source JavaScript library for mobile-friendly interactive maps.
This notebook additionally uses [Leafmap](leafmap.org) to expose this functionality within the jupyter environment
This library enables interactive maps in the Jupyter notebook/JupyterLab environment.

## Description

This notebook demonstrates how to visualise timeseries data in an interactive manner

---

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages

In [10]:
import os
import numpy as np
from ipywidgets import widgets as w
from IPython.display import display
import leafmap
import datacube
import odc.ui
import pandas as pd
import odc.geo.xr
from ipyleaflet import ImageOverlay

### Connect to the datacube

In [11]:
dc = datacube.Datacube(app='Imagery_on_web_map')

In [12]:
def da_to_timeseries_imageoverlay(data):
    """
    Converts a xarray.DataArray into a dictionary with times as keys and ipyleaflet ImageOverlays as values.
    
    Parameters
    ----------
    da : xarray dataarray or a numpy ndarray  
    
    Returns
    -------
    out : dictionary
    bounds: bounds of input da
    
    """
    out = {}
    for time in data.time.values:
        date = str(pd.to_datetime(time).date())
        print('\r', 'Opening:', date, end='')
        single = data.sel(time=time)
        vmin, vmax = np.nanpercentile(data.data, [2, 98])
        rgba = single.odc.colorize(vmin=vmin, vmax=vmax)
        data_url = rgba.odc.compress(as_data_url=True)
        (x1, y1), _, (x2, y2) = rgba.odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
        bounds = [[y1, x1], [y2, x2]]
        raster = ImageOverlay(url=data_url, bounds=bounds,opacity=0.7)
        out[date] = raster
    return out, bounds

In [13]:
# Set up a region to load data
query = {
    'y': (-34.06813, -34.22099),
    'x': (139.83315, 140.05151),
    'time': ('2000-08-31', '2000-10-31'),
}

# Load DEA Fractional Cover data from the datacube
fc = dc.load(product='ga_ls_fc_3',
             measurements=['pv'],
             output_crs='EPSG:32754',
             resolution=(-30, 30),
             group_by='solar_day',
             progress_cbk=odc.ui.with_ui_cbk(),
             **query)

In [14]:
time_dict, bounds = da_to_timeseries_imageoverlay(fc['pv'])

 Opening: 2000-10-26

In [15]:
# Create leafmap approximately centered using bounds
m = leafmap.Map(center = bounds[0],
               zoom=11)

In [16]:
# Create dictionary of images for time slider
m.add_time_slider(time_dict)

In [17]:
# Display map
display(m)

Map(center=[-34.06619528124256, 139.83298969912786], controls=(ZoomControl(options=['position', 'zoom_in_text'…

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2021

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)